In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
import random
from itertools import chain
from collections import Counter
import numpy as np
from scipy.stats import norm
import math
from operator import add

from ipynb.fs.full.GenerateData import generatedata


**0) Define variables**

$ \{x_{1}, x_{2},...x_{k}\} \in X$, Observed features of defendant
 
$\mu_{prior}$, Judge's mean prior belief about a defendant; $\textit{most probable}$ risk prediction

$\sigma_{prior}$, Standard deviation of judge's prior belief on a defendant; corresponds to a judge's $\textit{uncertainty}$ about prediction

$\mu_{ra}$, Observed algorithmic risk assessment prediction; mean of normally-distributed perceived anchor information

$\sigma_{ra}$, Perceived anchor $\textit{influence}$; Corresponds to S.D. of risk assessment (when perceived as containing a lot of information S.D. will be lower and vice versa).

$q$, Parameter mapping $\mu_{prior},\sigma_{prior}$ onto $\mu_{post},\sigma_{post}$

$\Theta$, Threshold distance between $\mu_{prior}$ and $\mu_{ra}$ after which $\mu_{ra}$ has no (or minimal) effect on $\mu_{prior}$

$\mu_{post}$, Judge's mean posterior belief

$\sigma_{post}$, Standard deviation of judge's posterior belief 

$\tau$, Decision making threshold

$\phi_{prior}(\tau;\mu_{prior},\sigma_{prior})$, Gaussian distribution representing judge's prior belief

$\phi_{post}(\tau;\mu_{post},\sigma_{post})$, Gaussian distribution representing the judge's posterior belief

$\Phi_{post}(\tau;\mu_{post},\sigma_{post})$, Probability of drawing a belief from the posterior greater than threshold $\tau$

$y$, Judge's observed decision on defendant
 
$\alpha$, Learning rate


**1) Helper functions**

**(a) Initialize parameters**

$\beta_{1}$ (nx1), $\beta_{2}$ (nx1), and $w$ (1x1) are randomly initialized (>0);
$b$ (1x1) and $c$ (1x1) are initialized as zero

In [87]:
def initialize_parameters(X,lower,upper):
    """
    Argument:
    
    Returns:

    """
    random.seed(22)#2304
    
    # of parameters
    n = X.shape[1]
    
    B = [random.random() for i in range(n)]
    b = random.random()*10
    q = random.random()
    sd = random.random()*10
    tau = random.randint(lower, upper)
    Theta = random.randint(lower, upper)
    scale = 25
    
    parameters = {"B": np.array([B]).T,
                  "b": b,
                  "q" : q,
                  "sd_prior" : sd,
                  "tau" : tau,
                  "Theta" : Theta,
                  "scale": scale}
    
    return parameters

In [3]:
# create dictionary to store derivatives
def initialize_dicts():
    derivatives = {
        "dL_dphi": [],
        "dphipost_dmupost": [],
        "dphipost_dsdpost": [],
        "dphipost_dtau": [],
        "dmupost_dmuprior": [],
        "dsdpost_dq": [],
        "dmupost_dq": [],
        "dsdpost_dsdprior": [],
        "dmupost_dI" : [],
        "dsdpost_dI" : [],
        "dI_dd" : [],
        "dd_dmuprior" : [],
        "dd_dtheta" : []
    }

    grads = {
        "dB" : [],
        "db" : [],
        "dsdprior" : [],
        "dq" : [],
        "dtau" : [],
        "dTheta" : []
    }
    
    return derivatives,grads

**2) Forward**

**(a) Estimate mean of prior belief distribution**

Given {$\beta, b, \sigma, \theta, \tau, w$}, want to calculate loss, $\mathcal{L}(\hat{y},y_i)$ using the following steps:

1. Calculate prior mean, $\mu_{prior}$
\begin{equation}
\mu_{prior} = \beta X + b
\end{equation}

Initialize the first estimated prior mean and store in the cache. 
Future calculations will use the function calc_prior_mean

In [4]:
def calc_prior_mean(X,parameters):
    """
    Arguments:
    
    Returns:
    """
    
    B = parameters['B']
    b = parameters['b']
    
    mu_prior = np.dot(B.T,X.T)+b
    
    return mu_prior[0].tolist()

**(b) Estimate the standard deviation of the risk assessment score (i.e. perceived confidence in the anchor info):**

**We're not currently including this in further calculations**

\begin{equation}
\sigma_{ra}^2 = var_{ra} = \left \{
	\begin{array}{ll}
        q \cdot var_{prior} & if |\mu_{ra} - \mu_{prior}|<\Theta \\
        \infty & otherwise
    \end{array}
    \right.
\end{equation}

In [5]:
#def calc_var_ra(sd_prior,mu_prior,mu_ra,q,theta): 
#    """
#    #Arguments:

#    #Returns:
#    """

#    var_ra = [q*sd_prior if np.abs(mu_ra[i]-mu_prior[i])<theta else float("inf") for i in range(len(mu_prior))]

#    return var_ra 

**(c) Calculate the posterior distribution of a judge's beliefs on defendant, $\phi_{post}(\mu_{post},\sigma_{post}^2)$, and probability of making decision, $\hat{y}$:**



\begin{equation}
\phi_{prior}(\mu_{prior},\sigma_{prior}^2) = \mathcal{N}(\mu_{prior},\sigma_{prior}^2)
\end{equation}


\begin{equation}
    \phi_{ra}(\mu_{ra},\sigma_{ra}^2) = \mathcal{N}(\mu_{ra},\sigma_{ra}^2)
\end{equation}

\begin{equation}
\phi_{post}(\mu_{post},\sigma_{post}^2) =  \mathcal{N}(\mu_{post},\sigma_{post}^2)
\end{equation}

\begin{equation}
    = \frac{\phi_{prior}(\mu_{prior},\sigma_{prior}^2)\phi_{ra}(\mu_{ra},\sigma_{ra}^2)}{\int \phi_{prior}(\mu_{prior},\sigma_{prior}^2)\phi_{ra}(\mu_{ra},\sigma_{ra}^2)}
\end{equation}

where:

\begin{equation}
d = |\mu_{ra}-\mu_{prior}|-\Theta
\end{equation}

\begin{equation}
I = 1 - \frac{1}{1+e^{-d*scale}}
\end{equation}

\begin{equation}
\mu_{post} = I\cdot\left(\mu_{prior} \cdot \frac{q}{q+1} + \mu_{ra} \cdot \frac{1}{q+1}\right) + (1-I)\cdot\mu_{prior}
\end{equation}

\begin{equation}
\sigma_{post} = I\cdot\left(\sigma_{prior} \cdot \sqrt{\frac{q}{q+1}}\right) + (1-I)\cdot\sigma_{prior}
\end{equation}

In [20]:
#test = list(range(-10,10))
#test_sig = [1-(1/(1+np.exp(-25*t))) for t in test]
#plt.plot(test,test_sig)

In [ ]:
def calc_post_mean(mu_prior, mu_ra, parameters):
    """
    Arguments:

    Returns:
    """
    theta = parameters['Theta']
    q = parameters['q']
    scale = parameters['scale']
    
    difference = [np.abs(mu_ra[i]-mu_prior[i])-theta for i in range(len(mu_prior))]
    I = [1-(1/(1+np.exp(-scale*d))) for d in difference]
    mu_post = [(I[i]*((mu_prior[i]*(q/(q+1)))+(mu_ra[i]/(q+1))))+((1-I[i])*mu_prior[i]) for i in range(len(mu_prior))]
    
    return mu_post

In [18]:
def calc_post_sd(mu_prior, mu_ra, parameters):
    """
    Arguments:
    
    Returns:
    """
    q = parameters['q']
    sd_prior = parameters['sd_prior']
    theta = parameters['Theta']
    scale = parameters['scale']
    
    difference = [np.abs(mu_ra[i]-mu_prior[i])-theta for i in range(len(mu_prior))]
    I = [1-(1/(1+np.exp(-scale*d))) for d in difference]
    sd_post = [(I[i]*(sd_prior*np.sqrt(q/(q+1))))+((1-I[i])*sd_prior) for i in range(len(mu_prior))]
    
    return sd_post

**(d) Calculate the probability of detaining defendant as area under the posterior belief curve $\geq$ decision threshold, $\tau$**

\begin{equation}
\phi(\tau; \mu_{post},\sigma_{post}) = \mathcal{N}(\tau;\mu_{post},\sigma_{post}) \\
\Phi(\tau;\mu_{post},\sigma_{post}) = \int_\tau^{\infty} \phi_{post}(\tau; \mu_{post},\sigma_{post}) \\
\hat{y}=1-\Phi(\tau;\mu_{post},\sigma_{post}) 
\end{equation}


In [ ]:
def calc_y(mu_post,sd_post,parameters):
    """
    Arguments:
    
    Returns:
    """
    tau = parameters['tau']

    y_pred = [1 - norm.cdf(tau,loc=mu_post[i],scale=sd_post[i]) for i in range(len(mu_post))]
    
    y_pred = [1e-13 if p==0 else p for p in y_pred]
    y_pred = [1-1e-13 if p==1 else p for p in y_pred]

    return y_pred

**(e) Compute negative log likelihood**


\begin{equation}
\mathcal{L}(\hat{y}, y_i) = \frac{1}{m}\sum_{1=1}^{m}- y_i \log (\hat{y}) - (1-y_i) \log (1-\hat{y})
\end{equation}


In [20]:
def calc_L(y_pred,Y):
    """
    Arguments:

    Returns:
    """
    loss = 0
    for i in range(len(y_pred)):
        loss += -(Y[i]*np.log(y_pred[i]))-((1-Y[i])*np.log(1-y_pred[i]))

    L = loss/len(y_pred)
    
    return L

**4) Calculate derivatives for gradient descent**

\begin{equation}
    \frac{d\mathcal{L}}{d\phi_{post}} = -1 \cdot \phi_{post} \cdot \left(-\frac{y}{\hat{y}}+\frac{1-y}{1-\hat{y}}\right)
\end{equation}

\begin{equation}
    \frac{d\phi_{post}}{d\mu_{post}} = \frac{\tau-\mu_{post}}{\sigma_{post}^3\cdot\sqrt{2\pi}}\cdot exp \left(- \frac{(\tau-\mu_{post})^2}{2\cdot \sigma_{post}^2} \right)
\end{equation}

\begin{equation}
    \frac{d\phi_{post}}{d\sigma_{post}} = \left(\frac{(\tau-\mu_{post})^2}{\sigma_{post}^4\sqrt{2\pi}} - \frac{1}{\sigma_{post}^2 \sqrt{2\pi}} \right) \cdot exp \left[- \frac{(\tau-\mu_{post})^2}{2\cdot \sigma_{post}^2} \right]
\end{equation}

\begin{equation}
    \frac{d\phi_{post}}{d\tau}= -\frac{\tau-\mu_{post}}{\sigma_{post}^3 \sqrt{2\pi}}exp\left[-\frac{(\tau-\mu_{post})^2}{2\sigma_{post}^3} \right]
\end{equation}

\begin{equation}
    \frac{d\mu_{post}}{d\mu_{prior}} = 
    1-I+\frac{I\cdot q}{q+1}
\end{equation}

\begin{equation}
    \frac{d\mu_{post}}{dq} = I\cdot\left(\frac{\mu_{prior}}{1+q} - \frac{\mu_{ra}}{(1+q)^2}-\frac{q\cdot\mu_{prior}}{(1+q)^2}\right)
\end{equation}        

\begin{equation}
    \frac{d\sigma_{post}}{dq} = I\cdot \sigma_{prior} \left( \frac{-\frac{q}{(1+q)^2} + \frac{1}{1+q}}{2\sqrt{\frac{q}{q+1}}} \right)
\end{equation}

\begin{equation}
    \frac{d\sigma_{post}}{d\sigma_{prior}} = 1-I+I\cdot\sqrt{\frac{q}{q+1}}
\end{equation}

\begin{equation}
\frac{d\mu_{prior}}{d\beta} = X
\end{equation}

\begin{equation}
\frac{d\mu_{prior}}{db} = 1
\end{equation}

\begin{equation}
    \frac{d\mu_{post}}{dI} = -\mu_{prior}+\frac{\mu_{ra}}{q+1}+\frac{\mu_{prior}\cdot q}{q+1}
\end{equation}

\begin{equation}
    \frac{d\sigma_{post}}{dI} = -\sigma_{prior}+\sigma_{prior}\cdot\sqrt{\frac{q}{q+1}}
\end{equation}

\begin{equation}
    \frac{dI}{dd} = - \frac{scale \cdot e^{-d \cdot scale}}{(1+e^{-d \cdot scale})^2} 
\end{equation}

\begin{equation}
    \frac{dd}{d\mu_{prior}} =
    \left \{
	\begin{array}{ll}
        -1 & if \mu_{ra} - \mu_{prior}>0 \\
        1 & if \mu_{ra} - \mu_{prior}<0
    \end{array}
    \right.
\end{equation}

\begin{equation}
    \frac{dd}{d\Theta} = -1
\end{equation}

In [1]:
def calc_component_derivs(X, parameters, derivatives, mu_prior, mu_ra, mu_post, sd_post, y_pred, Y):
        
    B = parameters['B']
    b = parameters['b']
    q = parameters['q']
    sd_prior = parameters['sd_prior']
    tau = parameters['tau']
    Theta = parameters['Theta']
    scale = parameters['scale']
    
    difference = [np.abs(mu_ra[i]-mu_prior[i])-Theta for i in range(len(mu_prior))]
    Indicator = [1-(1/(1+np.exp(-1*d*scale))) for d in difference]
    
    for i in range(len(X)):
        
        #retrieve variables
        s_post = sd_post[i]
        y = Y[i]
        m_prior = mu_prior[i]
        m_post = mu_post[i]
        m_ra = mu_ra[i]
        y_p = y_pred[i]
        d_0 = difference[i]
        I = Indicator[i]
        
        # calculate derivatives / partial derivatives
    
        # a few expressions for reuse
        exponent = np.exp(-((tau-m_post)**2)/(2*(s_post**2)))

        # dL/dphi
        phi_post = (1/(s_post*np.sqrt(2*math.pi)))*exponent
        dL_dphi = -1*phi_post*(((1-y)/(1-y_p))-(-y/y_p))

        # dphipost_dmupost
        dphipost_dmupost = ((tau-m_post)/((s_post**3)*np.sqrt(2*math.pi)))*exponent 

        # dphipost_dsdpost
        dphi_dsd_1 = ((tau-m_post)**2)/((s_post**4)*np.sqrt(2*math.pi))
        dphi_dsd_2 = 1/((s_post**2)*np.sqrt(2*math.pi))
        dphipost_dsdpost = (dphi_dsd_1-dphi_dsd_2)*exponent

        # dphipost_dtau
        dphipost_dtau = -((tau-m_post)/((s_post**3)*np.sqrt(2*math.pi)))*exponent

        # dmupost_dmuprior
        dmupost_dmuprior = 1-I+((I*q)/(q+1))
        
        # dmupost_dq
        dmupost_dq = I*((m_prior/(1+q)) - (m_ra/((q+1)**2)) - ((m_prior*q)/((q+1)**2)))
        
        # dsdpost_dq
        dsdpost_dq_num = -(q/((1+q)**2))+(1/(1+q))
        dsdpost_dq_den = 2*np.sqrt(q/(1+q))
        dsdpost_dq = I*sd_prior*(dsdpost_dq_num/dsdpost_dq_den)
        
        # dsdpost_dsdprior
        dsdpost_dsdprior = 1-I+(I*np.sqrt(q/(q+1)))
        
        # dmupost_dI
        dmupost_dI = -m_prior + (m_ra/(q+1)) + ((m_prior*q)/(q+1))
        
        # dsdpost_dI
        dsdpost_dI = -sd_prior+(sd_prior*np.sqrt(q/(q+1)))
        
        # dI_dd
        dI_dd = -(scale*np.exp(-1*scale*d_0))/((1+np.exp(-1*scale*d_0))**2)
        
        # dd_dmuprior
        if m_ra-m_prior>0:
            dd_dmuprior = -1
        else:
            dd_dmuprior = 1

        # dd_dtheta
        dd_dtheta = -1

        derivatives['dL_dphi'].append(dL_dphi)
        derivatives['dphipost_dmupost'].append(dphipost_dmupost)
        derivatives['dphipost_dsdpost'].append(dphipost_dsdpost)
        derivatives['dphipost_dtau'].append(dphipost_dtau)
        derivatives['dmupost_dmuprior'].append(dmupost_dmuprior)
        derivatives['dsdpost_dq'].append(dsdpost_dq)
        derivatives['dmupost_dq'].append(dmupost_dq)
        derivatives['dsdpost_dsdprior'].append(dsdpost_dsdprior)
        derivatives['dmupost_dI'].append(dmupost_dI)
        derivatives['dsdpost_dI'].append(dsdpost_dI)
        derivatives['dI_dd'].append(dI_dd)
        derivatives['dd_dmuprior'].append(dd_dmuprior)
        derivatives['dd_dtheta'].append(dd_dtheta)

    
    return derivatives


**5) Use chain rule to calculate gradients**

\begin{equation}
    \frac{d\mathcal{L}}{d\beta} =  \frac{1}{m}\sum_{1=1}^{m}\frac{d\mathcal{L}}{d\phi_{post}}\cdot \frac{d\phi_{post}}{d\sigma_{post}} \cdot \frac{d\sigma_{post}}{dI} \cdot \frac{dI}{dd} \cdot \frac{dd}{d\mu_{prior}} \cdot \frac{d\mu_{prior}}{d\beta} + \frac{d\mathcal{L}}{d\phi_{post}} \cdot \frac{d\phi_{post}}{d\mu_{post}} \cdot \frac{d\mu_{post}}{d\mu_{prior}} \cdot \frac{d\mu_{prior}}{d\beta} + \frac{d\mathcal{L}}{d\phi_{post}} \cdot \frac{d\phi_{post}}{d\mu_{post}} \cdot \frac{d\mu_{post}}{dI} \cdot \frac{dI}{dd} \cdot \frac{dd}{d\mu_{prior}} \cdot \frac{d\mu_{prior}}{d\beta}
\end{equation}

\begin{equation}
    \frac{d\mathcal{L}}{db} =  \frac{1}{m}\sum_{1=1}^{m}\frac{d\mathcal{L}}{d\phi_{post}}\cdot \frac{d\phi_{post}}{d\sigma_{post}} \cdot \frac{d\sigma_{post}}{dI} \cdot \frac{dI}{dd} \cdot \frac{dd}{d\mu_{prior}} \cdot \frac{d\mu_{prior}}{db} + \frac{d\mathcal{L}}{d\phi_{post}} \cdot \frac{d\phi_{post}}{d\mu_{post}} \cdot \frac{d\mu_{post}}{d\mu_{prior}} \cdot \frac{d\mu_{prior}}{db} + \frac{d\mathcal{L}}{d\phi_{post}} \cdot \frac{d\phi_{post}}{d\mu_{post}} \cdot \frac{d\mu_{post}}{dI} \cdot \frac{dI}{dd} \cdot \frac{dd}{d\mu_{prior}} \cdot \frac{d\mu_{prior}}{db}
\end{equation}

\begin{equation}
    \frac{d\mathcal{L}}{d\sigma_{prior}} = \frac{1}{m}\sum_{1=1}^{m}\frac{d\mathcal{L}}{d\phi_{post}}\cdot\frac{d\phi_{post}}{d\sigma_{post}} \cdot \frac{d\sigma_{post}}{d\sigma_{prior}} 
\end{equation}

\begin{equation}
    \frac{d\mathcal{L}}{dq} = \frac{1}{m}\sum_{1=1}^{m}\frac{d\mathcal{L}}{d\phi_{post}}\cdot\frac{d\phi_{post}}{d\sigma_{post}} \cdot \frac{d\sigma_{post}}{dq} + \frac{d\mathcal{L}}{d\phi_{post}}\cdot\frac{d\phi_{post}}{d\mu_{post}} \cdot \frac{d\mu_{post}}{dq} 
\end{equation}

\begin{equation}
    \frac{d\mathcal{L}}{d\tau} = \frac{1}{m}\sum_{1=1}^{m}\frac{d\mathcal{L}}{d\phi_{post}}\cdot\frac{d\phi_{post}}{d\tau}
\end{equation}

\begin{equation}
\frac{d\mathcal{L}}{d\Theta} = \frac{1}{m}\sum_{1=1}^{m}\frac{d\mathcal{L}}{d\phi_{post}}\cdot\frac{\partial\phi_{post}}{\partial\mu_{post}}\cdot \frac{\partial\mu_{post}}{\partial I}\cdot \frac{dI}{dd} \cdot \frac{dd}{d\Theta}+\frac{d\mathcal{L}}{d\phi_{post}}\cdot\frac{\partial\phi_{post}}{\partial\sigma_{post}}\cdot \frac{\partial\sigma_{post}}{\partial I} \cdot \frac{\partial I}{\partial dd} \cdot \frac{dd}{d\Theta}
\end{equation}

In [46]:
def calc_gradients(X,grads,derivatives): 
    
    n = X.shape[1]
    B_grads = [0]*n
    b_grads = 0
    q_grads = 0
    tau_grads = 0
    sd_prior_grads = 0
    theta_grads = 0
    
    for i in range(len(X)):
        # Retrieve variables
        dL_dphipost = derivatives['dL_dphi'][i]
        dphipost_dmupost = derivatives['dphipost_dmupost'][i]
        dphipost_dsdpost = derivatives['dphipost_dsdpost'][i]
        dphipost_dtau = derivatives['dphipost_dtau'][i]
        dmupost_dmuprior = derivatives['dmupost_dmuprior'][i]
        dsdpost_dq = derivatives['dsdpost_dq'][i]
        dmupost_dq = derivatives['dmupost_dq'][i]
        dsdpost_dsdprior = derivatives['dsdpost_dsdprior'][i]
        dmupost_dI = derivatives['dmupost_dI'][i]
        dsdpost_dI = derivatives['dsdpost_dI'][i]
        dI_dd = derivatives['dI_dd'][i]
        dd_dmuprior = derivatives['dd_dmuprior'][i]
        dd_dtheta = derivatives['dd_dtheta'][i]
        x = X.iloc[i,:].tolist()

        # dL_dB
        B_grads = [sum(z) for z in zip(B_grads,np.dot(dL_dphipost*dphipost_dsdpost*dsdpost_dI*dI_dd*dd_dmuprior,x)+np.dot(dL_dphipost*dphipost_dmupost*dmupost_dmuprior,x)+np.dot(dL_dphipost*dphipost_dmupost*dmupost_dI*dI_dd*dd_dmuprior,x))]

        # dL_db
        b_grads += (dL_dphipost*dphipost_dsdpost*dsdpost_dI*dI_dd*dd_dmuprior*1) + (dL_dphipost*dphipost_dmupost*dmupost_dmuprior*1) + (dL_dphipost*dphipost_dmupost*dmupost_dI*dI_dd*dd_dmuprior*1)

        #dL_dsdprior
        sd_prior_grads += dL_dphipost*dphipost_dsdpost*dsdpost_dsdprior

        # dL_dq
        q_grads += (dL_dphipost*dphipost_dsdpost*dsdpost_dq) + (dL_dphipost*dphipost_dmupost*dmupost_dq)

        # dL_dtau
        tau_grads += dL_dphipost*dphipost_dtau
        
        # dL_dTheta
        theta_grads += (dL_dphipost*dphipost_dmupost*dmupost_dI*dI_dd*dd_dtheta) + (dL_dphipost*dphipost_dsdpost*dsdpost_dI*dI_dd*dd_dtheta)
    
    grads['dB'] = [gr/float(len(X)) for gr in B_grads]
    grads['db'] = b_grads/float(len(X))
    grads['dsdprior'] = sd_prior_grads/float(len(X))
    grads['dq'] = q_grads/float(len(X))
    grads['dtau'] = tau_grads/float(len(X))
    grads['dTheta'] = theta_grads/float(len(X))
    
    return grads

**8) Update parameters with gradients**

\begin{equation}
\beta' = \beta - \alpha \frac{d\mathcal{L}}{d\beta} \\
b' = b - \alpha \frac{d\mathcal{L}}{db} \\
q' = q - \alpha \frac{d\mathcal{L}}{dq} \\
var_{prior}' = var_{prior}-\alpha\frac{d\mathcal{L}}{dvar_{prior}} \\
\tau' = \tau - \alpha \frac{dL}{d\tau} \\
\Theta' = \Theta - \alpha \frac{dL}{d\Theta}
\end{equation}

In [23]:
def update_parameters(parameters, grads, learning_rate):
    """
    Arguments:
    
    Returns:
    parameters -- dictionary containing updated parameters 
    """
    #parameters['B'] = np.array([(B-(learning_rate*gB)).tolist() for gB,B in zip(grads['dB'],parameters['B'])])
    #parameters['b'] -= learning_rate*grads['db']
    parameters['q'] -= learning_rate*grads['dq']
    #parameters['sd_prior'] -= learning_rate*grads['dsdprior']
    #parameters['tau'] -= learning_rate*grads['dtau']
    #parameters['Theta'] -= learning_rate*grads['dTheta']
    
    return parameters

In [24]:
def predict_choice(X_test,risk_assess_test,best_params):
    """
    Arguments:
    
    Returns:
    """
    mu_prior_test = calc_prior_mean(X_test, best_params)
    mu_post_test = calc_post_mean(mu_prior_test, risk_assess_test, best_params)
    sd_post_test =calc_post_sd(mu_prior_test, risk_assess_test, best_params)
    y_test = calc_y(mu_post_test, sd_post_test, best_params)
    
    return y_test